In [136]:
import xml.etree.ElementTree as ET, sys, urllib.request, json
if 'functions' in sys.modules:
    del sys.modules['functions'] # deletes previously imported module so that potential changes can be loaded
from functions import *

In [74]:
# The XML url's for every single protocol are grouped in packages of 5 whose url's are put in a dataloader. get the dataloader url.
# There are three data-loader-urls. take the first one for the current Wahlperiode.
url = 'https://www.bundestag.de/services/opendata'
htmlString = get(url).text
soup = BeautifulSoup(htmlString, 'lxml')
dataloader_url = [x.attrs['data-dataloader-url'] for x in soup.find_all('div', {'class':'bt-module-row-dokumente'})][0]

In [70]:
%%time
# Get the single XML url's (runtime ~ 1.5 min)
xml_urls = []
home = 'https://www.bundestag.de'
i = 0
while True:
    url_package_i = home + data_loader_url + '?offset=%i' %i
    htmlString = get(url_package_i).text
    soup = BeautifulSoup(htmlString, 'lxml')
    xml_urls_i = [home + x.attrs['href'] for x in soup.find_all('a', {'class':'bt-link-dokument'})]
    if len(xml_urls_i) == 0:
        break
    xml_urls += xml_urls_i
    i += 5

CPU times: user 468 ms, sys: 16 ms, total: 484 ms
Wall time: 1min 20s


In [137]:
%%time
# parse XML files (runtime ~ 6 min)
redner, reden = {}, {}
for url in xml_urls:
    root = parse_xml(url)
    redner = get_redner_dict(root, redner)
    reden = get_reden(root, reden)

CPU times: user 6.63 s, sys: 316 ms, total: 6.94 s
Wall time: 5min 36s


In [153]:
# save dicts
with open('../data/bundestag/redner.json', 'w') as f:
    json.dump(redner, f)
with open('../data/bundestag/reden.json', 'w') as f:
    json.dump(reden, f)